In [7]:

import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from torch.utils.data import DataLoader

from model_init import *
from dataset_init import *
from utils.others import *
from utils.testModel import *

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Current Device " , device)

batch_size = 64
num_epochs = 1000
num_users = 3
nb_classes = 10
dataset = 'MNIST'

Current Device  cuda


In [9]:
clean_dataset, unlearning_dataset, user_wm_dataset, test_dataset = unlearning_dataset_init(dataset, 3)
print(len(user_wm_dataset))

6


In [10]:
train_loader = DataLoader(unlearning_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [11]:
D, G = model_init(dataset, device)
D.load_state_dict(torch.load('./D-CL.pth'))
optimizerD = torch.optim.Adam(filter(lambda p: p.requires_grad, D.parameters()), 0.0002, betas=(0.5, 0.999))
optimizerG = torch.optim.Adam(G.parameters(), 0.0002, betas=(0.5, 0.999))
criterion_adv = nn.BCELoss()
criterion_aux = nn.CrossEntropyLoss() 

In [12]:
total_step = len(train_loader)
comprehensive_user_test(D, device, test_loader, user_wm_dataset)
for epoch in range(num_epochs):
    for batch_idx, (images, target) in enumerate(train_loader):
        images, target = images.to(device), torch.LongTensor(target).to(device)
        predictR, predictRLabel = D(images) #image from the real dataset
        loss_real_aux = criterion_aux(predictRLabel, target)
        optimizerD.zero_grad()
        optimizerG.zero_grad()
        loss_real_aux.backward()
        optimizerD.step()
        real_score = predictR
    comprehensive_user_test(D, device, test_loader, user_wm_dataset)


Testing on Users' datasset
Testing on User 0 watermark:

Test set: Average loss: 0.0009, Accuracy: 967/967 (100%)

Testing on User 1 watermark:

Test set: Average loss: 0.0025, Accuracy: 1138/1138 (100%)

Testing on User 2 watermark:

Test set: Average loss: 0.0183, Accuracy: 998/1003 (100%)

Testing on User 3 watermark:

Test set: Average loss: 0.0040, Accuracy: 1025/1027 (100%)

Testing on User 4 watermark:

Test set: Average loss: 0.0227, Accuracy: 994/1000 (99%)

Testing on User 5 watermark:

Test set: Average loss: 0.0129, Accuracy: 873/873 (100%)

Testing on clean test set


KeyboardInterrupt: 